This work is infludenced by [this notebook.](https://github.com/TheKidPadra/DeepLearning.AI-TensorFlow_Developer-specialization/blob/main/3.%20Natural%20Language%20Processing%20in%20TensorFlow/W4/ungraded_labs/C3_W4_Lab_1.ipynb)

In [1]:
#import libraries
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import random

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [2]:
data="In the town of Athy one Jeremy Lanigan \n Battered away til he hadnt a pound. \nHis father died and made him a man again \n Left him a farm and ten acres of ground. \nHe gave a grand party for friends and relations \nWho didnt forget him when come to the wall, \nAnd if youll but listen Ill make your eyes glisten \nOf the rows and the ructions of Lanigans Ball. \nMyself to be sure got free invitation, \nFor all the nice girls and boys I might ask, \nAnd just in a minute both friends and relations \nWere dancing round merry as bees round a cask. \nJudy ODaly, that nice little milliner, \nShe tipped me a wink for to give her a call, \nAnd I soon arrived with Peggy McGilligan \nJust in time for Lanigans Ball. \nThere were lashings of punch and wine for the ladies, \nPotatoes and cakes; there was bacon and tea, \nThere were the Nolans, Dolans, OGradys \nCourting the girls and dancing away. \nSongs they went round as plenty as water, \nThe harp that once sounded in Taras old hall,\nSweet Nelly Gray and The Rat Catchers Daughter,\nAll singing together at Lanigans Ball. \nThey were doing all kinds of nonsensical polkas \nAll round the room in a whirligig. \nJulia and I, we banished their nonsense \nAnd tipped them the twist of a reel and a jig. \nAch mavrone, how the girls got all mad at me \nDanced til youd think the ceiling would fall. \nFor I spent three weeks at Brooks Academy \nLearning new steps for Lanigans Ball. \nThree long weeks I spent up in Dublin, \nThree long weeks to learn nothing at all,\n Three long weeks I spent up in Dublin, \nLearning new steps for Lanigans Ball. \nShe stepped out and I stepped in again, \nI stepped out and she stepped in again, \nShe stepped out and I stepped in again, \nLearning new steps for Lanigans Ball. \nBoys were all merry and the girls they were hearty \nAnd danced all around in couples and groups, \nTil an accident happened, young Terrance McCarthy \nPut his right leg through miss Finnertys hoops. \nPoor creature fainted and cried Meelia murther, \nCalled for her brothers and gathered them all. \nCarmody swore that hed go no further \nTil he had satisfaction at Lanigans Ball. \nIn the midst of the row miss Kerrigan fainted, \nHer cheeks at the same time as red as a rose. \nSome of the lads declared she was painted, \nShe took a small drop too much, I suppose. \nHer sweetheart, Ned Morgan, so powerful and able, \nWhen he saw his fair colleen stretched out by the wall, \nTore the left leg from under the table \nAnd smashed all the Chaneys at Lanigans Ball. \nBoys, oh boys, twas then there were runctions. \nMyself got a lick from big Phelim McHugh. \nI soon replied to his introduction \nAnd kicked up a terrible hullabaloo. \nOld Casey, the piper, was near being strangled. \nThey squeezed up his pipes, bellows, chanters and all. \nThe girls, in their ribbons, they got all entangled \nAnd that put an end to Lanigans Ball."

corpus = data.lower().split('\n')
print(corpus)

['in the town of athy one jeremy lanigan ', ' battered away til he hadnt a pound. ', 'his father died and made him a man again ', ' left him a farm and ten acres of ground. ', 'he gave a grand party for friends and relations ', 'who didnt forget him when come to the wall, ', 'and if youll but listen ill make your eyes glisten ', 'of the rows and the ructions of lanigans ball. ', 'myself to be sure got free invitation, ', 'for all the nice girls and boys i might ask, ', 'and just in a minute both friends and relations ', 'were dancing round merry as bees round a cask. ', 'judy odaly, that nice little milliner, ', 'she tipped me a wink for to give her a call, ', 'and i soon arrived with peggy mcgilligan ', 'just in time for lanigans ball. ', 'there were lashings of punch and wine for the ladies, ', 'potatoes and cakes; there was bacon and tea, ', 'there were the nolans, dolans, ogradys ', 'courting the girls and dancing away. ', 'songs they went round as plenty as water, ', 'the harp tha

In [3]:
def create_text_generation_data(sentence_list):

  """
  Creates a text generation data from a sentence list

  Parameters
  ----------
  sentence_list: array like
    a list of sentences

  Returns
  xs: array like
    tokenized and padded words

  labels:
    corresponding word to predict

  tokenizer: layers.Tokenizer()
    Tokenizer used in the process

  """


  #initialize the tokenizer
  tokenizer = tf.keras.layers.TextVectorization()

  #adapt the tokenizer
  tokenizer.adapt(sentence_list)

  #get the words
  total_words = tokenizer.vocabulary_size()

  #build the text generation dataset

  input_sequences = []
  for line in sentence_list: #iterate through the sentences

    #tokenize the sentence
    tokens = tokenizer([line]).numpy()[0]

    #iterate through the tokens
    #we need at least one word to predict next word range -> 1
    for i in range(1,len(tokens)):
      n_gram_sequence = tokens[:i+1]
      #store the generated sequences
      input_sequences.append(n_gram_sequence)

  #find the longest sentence to arrange padding length
  max_length = max([len(x) for x in input_sequences])

  #pad sequences
  padded_sequences = tf.keras.utils.pad_sequences(input_sequences,maxlen = max_length)

  #split the training data and labels
  xs,labels = padded_sequences[:,:-1],padded_sequences[:,-1]

  return xs,labels,tokenizer,total_words,max_length


def define_callbacks(model):
  es = tf.keras.callbacks.EarlyStopping(patience = 5,verbose = 1, restore_best_weights = True,monitor = 'loss')
  mc = tf.keras.callbacks.ModelCheckpoint(filepath = f"./ModelCheckpoints/{model.name}.ckpt",
                                         save_best_only = True,
                                         save_weights_only = True,
                                          monitor = 'loss')
  tb = tf.keras.callbacks.TensorBoard(log_dir = f"./TensorboardLogs/{model.name}")

  return es,mc,tb

In [4]:
#create models dict
models = {}
histories = {}

#use the function
xs,labels,tokenizer,total_words,max_length = create_text_generation_data(corpus)

In [5]:
#set up the parameters
MODEL_NAME = 'model1'
HISTORY_NAME = 'history1'

#create the model
models[MODEL_NAME] = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = total_words,
                              output_dim = 128,
                              input_length = max_length-1),

    tf.keras.layers.Bidirectional(layers.LSTM(16)),
    layers.Dense(total_words,activation = 'softmax')],name = MODEL_NAME)


#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

#define callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(xs,labels,epochs = 100,
                                                 callbacks = [es,mc,tb],verbose = 1)


Epoch 1/100
15/15 [==============================] - 16s 37ms/step - loss: 5.5717 - accuracy: 0.0177
Epoch 2/100
15/15 [==============================] - 0s 30ms/step - loss: 5.5429 - accuracy: 0.0508
Epoch 3/100
15/15 [==============================] - 1s 39ms/step - loss: 5.4758 - accuracy: 0.0574
Epoch 4/100
15/15 [==============================] - 0s 27ms/step - loss: 5.3227 - accuracy: 0.0574
Epoch 5/100
15/15 [==============================] - 1s 35ms/step - loss: 5.1583 - accuracy: 0.0486
Epoch 6/100
15/15 [==============================] - 0s 34ms/step - loss: 5.0745 - accuracy: 0.0530
Epoch 7/100
15/15 [==============================] - 0s 30ms/step - loss: 5.0253 - accuracy: 0.0751
Epoch 8/100
15/15 [==============================] - 0s 26ms/step - loss: 4.9827 - accuracy: 0.0662
Epoch 9/100
15/15 [==============================] - 0s 14ms/step - loss: 4.9434 - accuracy: 0.0640
Epoch 10/100
15/15 [==============================] - 0s 15ms/step - loss: 4.9050 - accuracy: 0.068

Increase LSTM layers

In [6]:
#set up the parameters
MODEL_NAME = 'model2'
HISTORY_NAME = 'history2'

#create the model
models[MODEL_NAME] = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = total_words,
                              output_dim = 128,
                              input_length = max_length-1),
    tf.keras.layers.Bidirectional(layers.LSTM(16,return_sequences = True)),
    tf.keras.layers.Bidirectional(layers.LSTM(16)),
    layers.Dense(total_words,activation = 'softmax')],name = MODEL_NAME)


#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

#define callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(xs,labels,epochs = 100,
                                                 callbacks = [es,mc,tb],verbose = 1)

Epoch 1/100
15/15 [==============================] - 11s 25ms/step - loss: 5.5730 - accuracy: 0.0221
Epoch 2/100
15/15 [==============================] - 0s 23ms/step - loss: 5.5475 - accuracy: 0.0574
Epoch 3/100
15/15 [==============================] - 0s 24ms/step - loss: 5.4539 - accuracy: 0.0508
Epoch 4/100
15/15 [==============================] - 0s 23ms/step - loss: 5.2556 - accuracy: 0.0508
Epoch 5/100
15/15 [==============================] - 0s 24ms/step - loss: 5.1248 - accuracy: 0.0508
Epoch 6/100
15/15 [==============================] - 0s 24ms/step - loss: 5.0755 - accuracy: 0.0530
Epoch 7/100
15/15 [==============================] - 0s 22ms/step - loss: 5.0396 - accuracy: 0.0486
Epoch 8/100
15/15 [==============================] - 0s 23ms/step - loss: 4.9890 - accuracy: 0.0574
Epoch 9/100
15/15 [==============================] - 0s 23ms/step - loss: 4.9486 - accuracy: 0.0574
Epoch 10/100
15/15 [==============================] - 0s 22ms/step - loss: 4.9067 - accuracy: 0.048

Increase the number of LSTM neurons

In [7]:
#set up the parameters
MODEL_NAME = 'model3'
HISTORY_NAME = 'history3'

#create the model
models[MODEL_NAME] = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = total_words,
                              output_dim = 128,
                              input_length = max_length-1),
    tf.keras.layers.Bidirectional(layers.LSTM(32)),
    layers.Dense(total_words,activation = 'softmax')],name = MODEL_NAME)


#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

#define callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(xs,labels,epochs = 100,
                                                 callbacks = [es,mc,tb],verbose = 1)

Epoch 1/100
15/15 [==============================] - 5s 18ms/step - loss: 5.5700 - accuracy: 0.0088
Epoch 2/100
15/15 [==============================] - 0s 15ms/step - loss: 5.5261 - accuracy: 0.0574
Epoch 3/100
15/15 [==============================] - 0s 17ms/step - loss: 5.3906 - accuracy: 0.0662
Epoch 4/100
15/15 [==============================] - 0s 16ms/step - loss: 5.1487 - accuracy: 0.0530
Epoch 5/100
15/15 [==============================] - 0s 27ms/step - loss: 5.0592 - accuracy: 0.0486
Epoch 6/100
15/15 [==============================] - 1s 43ms/step - loss: 5.0039 - accuracy: 0.0596
Epoch 7/100
15/15 [==============================] - 1s 39ms/step - loss: 4.9592 - accuracy: 0.0574
Epoch 8/100
15/15 [==============================] - 1s 47ms/step - loss: 4.8936 - accuracy: 0.0618
Epoch 9/100
15/15 [==============================] - 1s 43ms/step - loss: 4.8294 - accuracy: 0.0596
Epoch 10/100
15/15 [==============================] - 1s 40ms/step - loss: 4.7629 - accuracy: 0.0640

Add another dense layer.

In [8]:
#set up the parameters
MODEL_NAME = 'model4'
HISTORY_NAME = 'history4'

#create the model
models[MODEL_NAME] = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = total_words,
                              output_dim = 128,
                              input_length = max_length-1),
    tf.keras.layers.Bidirectional(layers.LSTM(32)),
    tf.keras.layers.Dense(total_words,activation = 'relu'),
    layers.Dense(total_words,activation = 'softmax')],name = MODEL_NAME)


#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

#define callbacks
es,mc,tb = define_callbacks(models[MODEL_NAME])

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(xs,labels,epochs = 100,
                                                 callbacks = [es,mc,tb],verbose = 1)

Epoch 1/100
15/15 [==============================] - 5s 33ms/step - loss: 5.5631 - accuracy: 0.0375
Epoch 2/100
15/15 [==============================] - 0s 31ms/step - loss: 5.4097 - accuracy: 0.0530
Epoch 3/100
15/15 [==============================] - 0s 31ms/step - loss: 5.1345 - accuracy: 0.0508
Epoch 4/100
15/15 [==============================] - 0s 31ms/step - loss: 5.0548 - accuracy: 0.0486
Epoch 5/100
15/15 [==============================] - 0s 18ms/step - loss: 5.0192 - accuracy: 0.0596
Epoch 6/100
15/15 [==============================] - 0s 18ms/step - loss: 4.9866 - accuracy: 0.0486
Epoch 7/100
15/15 [==============================] - 0s 18ms/step - loss: 4.9323 - accuracy: 0.0508
Epoch 8/100
15/15 [==============================] - 0s 18ms/step - loss: 4.8710 - accuracy: 0.0618
Epoch 9/100
15/15 [==============================] - 0s 17ms/step - loss: 4.8042 - accuracy: 0.0574
Epoch 10/100
15/15 [==============================] - 0s 18ms/step - loss: 4.6778 - accuracy: 0.0552

In [20]:
#!tensorboard dev upload --logdir /content/TensorboardLogs